In [37]:
from gurobipy import *
import snap
import numpy as np
import copy
import random
import matplotlib.pyplot as plt
import time

In [38]:
#load a graph G
node = 400
# G = np.loadtxt('test_10nodes.txt', delimiter='\t', skiprows=3, dtype="int")
# G = np.loadtxt('facebook_combined.txt', delimiter=' ', dtype="int")
# G = G.tolist()

FIn = snap.TFIn("large_acyclic.graph")
G1 = snap.TNGraph.Load(FIn)

# select k nodes
k = 50

In [39]:
#build up internal opinion s
s = np.loadtxt('internal_opinion_large_acyclic.txt', dtype="float")
s = s.tolist()

In [40]:
W = [[0]*node for i in range(node)]
for NI in G1.Nodes():
    for Id in NI.GetOutEdges():
        W[NI.GetId()][Id]=1
print(W)

#create list mapping to parent nodes for every nodes
# V = []
# for i in range(node):
#     V.append([])

# for i in range(len(W)):
#     temp =[]
#     for j in range(len(W[i])):
#         temp.append(W[i][j])
#     print(temp)
#     for a in range(len(temp)):
#         if temp[a] == 1:
#             V[i].append(a)
# print(V)

V = copy.deepcopy(W)
for i in  range(node):
    for j in range(len(W[i])):
        if W[i][j]==0:
            V[i][j]=99999
        else:
            V[i][j]=j
print(V)

#create b
temp = [0]*len(W)
for i in range(len(W)):
    if sum(W[i])==0:
        temp[i]=0
    else: 
        temp[i]=1
        
b=0
for i in range(len(temp)):
    if temp[i]==0:
        b+=1
print(b)

#create the list for how many neighbors for each nodes
N = [0]*len(W)
for i in range(len(W)):
    for j in range(len(W[i])):
        if W[i][j]==1:
            N[i]+=1
print(N)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [41]:
#create blank model
MILP_model = Model('Mixed Integer Linear Program')

#add variables
Z = MILP_model.addVars([(i) for i in range(node)],lb=0, ub=1, name="Z")
Y = MILP_model.addVars([(i) for i in range(node)] ,vtype=GRB.BINARY, name="Y")
X = MILP_model.addVars([(i,j) for i in range(node) for j in range(len(V[i]))] ,lb=0, ub=1, name="X")
MILP_model.update()

#add objective number
MILP_model.setObjective(quicksum(Z[i] for i in range(node)), GRB.MAXIMIZE)

#constraint 1
MILP_model.addConstr(quicksum(Y[i] for i in range(node)) <= k)
#constraint 2
MILP_model.addConstrs(Z[i]==Y[i]+s[i]-s[i]*Y[i] for i in range(0, b))
#constraint 3
MILP_model.addConstrs(Z[i] == 
                      Y[i]+
                      ((s[i]+quicksum(Z[j] for j in range(len(V[i])) if V[i][j] != 99999 ))/(N[i]+1))-
                      ((s[i]*Y[i])/(N[i]+1))-
                      (quicksum(X[i,j] for j in range(len(V[i])) if V[i][j] != 99999)/(N[i]+1)) for i in range(b,node))
#constraint 4
MILP_model.addConstrs(X[i,j]<=Z[j] for i in range(node) for j in range(len(V[i])) if V[i][j] != 99999)

# MILP_model.addConstrs(Y[i]<=Z[i] for i in range(node))

MILP_model.update()
MILP_model.write("debug.lp")

MILP_model.setParam("presolve",0)
MILP_model.setParam('SubMIPNodes',0)
MILP_model.setParam('Cuts',2)
MILP_model.setParam('MIRCuts',0)
MILP_model.setParam('Threads',1)

start = time.perf_counter()
MILP_model.optimize()
# MILP_model.presolve(0)
end = time.perf_counter()
print('time: ',end - start)

Changed value of parameter presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter SubMIPNodes to 0
   Prev: 500  Min: 0  Max: 2000000000  Default: 500
Changed value of parameter Cuts to 2
   Prev: -1  Min: -1  Max: 3  Default: -1
Changed value of parameter MIRCuts to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter Threads to 1
   Prev: 0  Min: 0  Max: 1024  Default: 0
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 1 threads
Optimize a model with 2276 rows, 160800 columns and 8700 nonzeros
Model fingerprint: 0x8dea7830
Variable types: 160400 continuous, 400 integer (400 binary)
Coefficient statistics:
  Matrix range     [8e-03, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-04, 5e+01]
Found heuristic solution: objective 219.1123715
Variable types: 160400 continuous, 400 integer (400 binary)

Root relaxation: objec

In [42]:
print(MILP_model.ObjVal)
# for var in MILP_model.getVars():
#     print(var.varName,'\t',var.x)

365.7070343935495


In [43]:
# MILP_model.printStats()

In [44]:
# MILP_model.printQuality()

In [45]:

# x = np.arange(1,k+1)
# plt.plot(x, gz_every_k,marker = 'o')
# plt.ylabel('g(z|T)')
# plt.xlabel('k')
# plt.xticks(range(1, k+1))
# plt.show()

In [46]:
365.3755578392006/365.7070343935495

0.9990936008247733